# STP 429 - Final Lab

## Meshach Samuel

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

sns.set()

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

### Importing Data

In [9]:
data = pd.read_excel("../Datasets/Fall 2024 Lab #3 Closing Prices.xlsx")
data.head()

,Date,ADP,ALLE,AME,AOS,AXON,BA,BLDR,BR,CARR,...,UAL,UBER,UNP,UPS,URI,VLTO,VRSK,WAB,WM,XYL
0,2023-11-01,210.369995,98.919998,142.399994,70.809998,207.149994,189.380005,117.470001,170.529999,49.660000,...,34.810001,43.830002,207.559998,139.800003,409.480011,67.510002,222.250000,105.800003,164.039993,94.480003
1,2023-11-02,218.559998,99.129997,143.699997,71.349998,210.270004,192.009995,121.250000,178.809998,49.970001,...,35.820000,46.480000,211.389999,141.990005,432.950012,70.279999,229.869995,109.349998,166.850006,94.309998
2,2023-11-03,219.309998,101.889999,143.699997,71.559998,217.250000,195.050003,126.790001,177.119995,50.529999,...,37.509998,47.750000,212.130005,142.860001,451.510010,70.800003,230.619995,111.290001,168.119995,96.930000
3,2023-11-06,220.539993,101.730003,143.460007,71.089996,216.860001,192.949997,125.589996,177.330002,50.270000,...,36.759998,48.139999,213.100006,142.270004,438.540009,68.260002,230.809998,110.269997,169.220001,96.160004
4,2023-11-07,222.250000,101.410004,146.500000,70.650002,216.369995,191.410004,126.059998,176.899994,50.450001,...,37.029999,49.919998,211.410004,141.850006,440.359985,70.070000,233.190002,109.800003,169.360001,96.769997


### Data Pre-Processing

#### Removing Unnecessary Columns

In [10]:
data = data.drop(["Date"], axis=1)
data.head()

,ADP,ALLE,AME,AOS,AXON,BA,BLDR,BR,CARR,CAT,...,UAL,UBER,UNP,UPS,URI,VLTO,VRSK,WAB,WM,XYL
0,210.369995,98.919998,142.399994,70.809998,207.149994,189.380005,117.470001,170.529999,49.660000,231.830002,...,34.810001,43.830002,207.559998,139.800003,409.480011,67.510002,222.250000,105.800003,164.039993,94.480003
1,218.559998,99.129997,143.699997,71.349998,210.270004,192.009995,121.250000,178.809998,49.970001,239.119995,...,35.820000,46.480000,211.389999,141.990005,432.950012,70.279999,229.869995,109.349998,166.850006,94.309998
2,219.309998,101.889999,143.699997,71.559998,217.250000,195.050003,126.790001,177.119995,50.529999,240.750000,...,37.509998,47.750000,212.130005,142.860001,451.510010,70.800003,230.619995,111.290001,168.119995,96.930000
3,220.539993,101.730003,143.460007,71.089996,216.860001,192.949997,125.589996,177.330002,50.270000,238.279999,...,36.759998,48.139999,213.100006,142.270004,438.540009,68.260002,230.809998,110.269997,169.220001,96.160004
4,222.250000,101.410004,146.500000,70.650002,216.369995,191.410004,126.059998,176.899994,50.450001,234.919998,...,37.029999,49.919998,211.410004,141.850006,440.359985,70.070000,233.190002,109.800003,169.360001,96.769997


### Linear Regression Model

#### EDA

Declaring Variables

In [11]:
y = data["DAL"]
x = data.drop(["DAL"], axis=1)